Run this notebook to download data from SafeGraph.

In [1]:
import pandas as pd
import sys, os
from csv import writer
sys.path.append('utils/')
import tqdm
#from census_tract_locator import *

In [2]:
import numpy as np
import json
import base64
import requests
from getpass import getpass
import geopandas as gpd
from shapely.geometry import Polygon

In [3]:
from helper_funcs import *
dirname = 'data/nyc_metro/'
createDir(dirname)
tract_file_path = '../Tracts/nyc_metro_boundaries/nyc_metro_boundaries.shp'

# this is used to filter the mobility rows that we want
boundary_data_for_filtering = gpd.read_file(tract_file_path)
boundary_data_for_filtering = boundary_data_for_filtering[['GEOID', 'state_abbr', 'geometry']]
boundary_data_for_filtering.GEOID = boundary_data_for_filtering.GEOID.astype(str)
boundary_data_for_filtering.head(2)

,GEOID,state_abbr,geometry
0,09003514500,CT,"POLYGON ((-72.56218 41.78077, -72.56122 41.780..."
1,09003514600,CT,"POLYGON ((-72.55995 41.77014, -72.55994 41.770..."


In [4]:
def get_tract_num(tract_data, point):
    if not tract_data['geometry'].contains(point).any():
        return np.nan
    tract_info = tract_data.loc[tract_data['geometry'].contains(point), ['GEOID']]
    tract_info = tract_info.iat[0, 0]
    print('found point inside')

    return tract_info

def get_geoids_and_filter(df, tract_data=boundary_data_for_filtering):
    df['GEOID'] = np.nan
    df.loc[df['poi_cbg'].notna(), 'poi_cbg'].apply(lambda x: print(f'CBG length incorrect: {x}') if len(x) != 12 else None)
    df.loc[df['poi_cbg'].notna(), 'GEOID'] = df.loc[df['poi_cbg'].notna(), 'poi_cbg'].apply(lambda x: x[0:-1])

    # get the tract that the POIs are located within
    cp = df.loc[df.GEOID.isna()]
    point = gpd.points_from_xy(x=cp['longitude'], y=cp['latitude'])
    points = gpd.GeoSeries(point, index=cp.index)

    cp['GEOID'] = points.apply(lambda p: get_tract_num(tract_data, p))
    df.loc[df.GEOID.isna(), 'GEOID'] = cp['GEOID']
    
    df.drop(index=df[df.GEOID.isna()].index, inplace=True)
    return df

In [5]:
boundary_data_for_filtering.state_abbr.unique()

array(['CT', 'PA', 'NY', 'NJ'], dtype=object)

In [6]:
def get_access_token(username, pword):
  
  credentials = f"{username}:{pword}" # Format credentials according to the API's expectations
  # print(f"Credentials: {credentials}")

  # Convert credentials to base64
  credentials_bytes = credentials.encode('ascii') 
  base64_credentials_bytes = base64.b64encode(credentials_bytes)
  base64_credentials = base64_credentials_bytes.decode('ascii')
  # print(f"Base64 credentials: {base64_credentials}")

  # Request access token
  headers = {
      'accept': 'application/json',
      'Authorization': f'Basic {base64_credentials}'
  }
  response = requests.post("https://marketplace.deweydata.io/api/auth/tks/get_token", headers=headers)
  response_access_token = response.json()['access_token']
  # print(f"Access token: {response_access_token}")

  return response_access_token


# Recursive function that returns the paths to each file available
def get_file_paths(token, path=""):
  headers = {
      'accept': 'application/json',
      'Authorization': f'Bearer {token}'
  }

  response = requests.get(f"https://marketplace.deweydata.io/api/data/v2/list/{path}", headers=headers)
  response_df = pd.DataFrame(response.json())
  if response_df.shape[0] > 0:
    files = response_df[response_df['directory']==False]
    folders = response_df[response_df['directory']==True]


    for name in folders.name.unique():
      new_files, new_folders = get_file_paths(token, path=f"{path}{name}/")
      files = pd.concat([files, new_files], sort=True)
      folders = pd.concat([folders, new_folders], sort=True)

    return files, folders
  return pd.DataFrame(), pd.DataFrame()


def read_file(data_url, cols, token, compression='gzip'):
  base_url = "https://marketplace.deweydata.io"
  full_url = f"{base_url}{data_url}"

  headers = {
      'accept': 'application/json',
      'Authorization': f'Bearer {token}'
  }

  df = pd.read_csv(full_url, storage_options=headers, compression='gzip', usecols=cols, dtype={'latitude':  np.float64, 'longitude':  np.float64, 'poi_cbg': str})

  return df

def process_dataframe(df):
    # drop if not in the correct state
    df.drop(index=df[~df.region.isin(boundary_data_for_filtering.state_abbr.unique())].index, inplace=True)

    # Drop row if there is a parent placekey (I checked and all parent placekeys do match up with a row placekey). Visits are attributed to parent AND children
    df.drop(index=df[df.parent_placekey.notna()].index, inplace=True)
    
    # drop if no visits
    df.drop(index=df[df['visitor_home_aggregation'].isna()].index, inplace=True)
    
    # get data in the tracts that i want
    df = get_geoids_and_filter(df)
    
    df = df.loc[df['GEOID'].isin(boundary_data_for_filtering.GEOID.unique())]
    return df

def request_files_and_save_to_drive(files_df, cols, token, month_counter, dir_path):
  for i, file_name in tqdm.tqdm(enumerate(files_df.url.unique())):
    tmp_df = read_file(file_name, cols, token)
    print(f'\nRead file {i}.')

    tmp_df = process_dataframe(tmp_df,)

    ah = os.path.exists(f'{dir_path}{month_counter}.csv')

    tmp_df.to_csv(f"{dir_path}{month_counter}.csv", mode='a', index=False, header=not os.path.exists(f'{dir_path}{month_counter}.csv'))

    
    print(f'Saved {tmp_df.shape[0]} rows.')
  
  print(f"--------done: month {month_counter}--------")
  return True

In [7]:
un = getpass('Enter your Dewey Marketplace username (email): ')
pw = getpass('Enter your Dewey Marketplace password: ')

Enter your Dewey Marketplace username (email): ········
Enter your Dewey Marketplace password: ········


In [8]:
access_token = get_access_token(un, pw)
print("done")
print(f"Access token: {access_token}")

rsp = get_file_paths(access_token)[0]

done
Access token: EV0eNuWPa9sxX453aExyxfl3Hhs


In [9]:
# for i, file_name in tqdm.tqdm(enumerate(spend_sample.url.unique())):
#     print(file_name)

In [10]:
cols=['placekey','parent_placekey', 'date_range_start', 'date_range_end', 'longitude', 'latitude', 'city', 'region', 'iso_country_code', 'visitor_home_aggregation' , 'poi_cbg']


for i, month_cd in enumerate(['07', '08', '09', '10', '11', '12']):
    spend_sample = rsp[rsp["parent"].str.endswith("SAFEGRAPH/WP")]
    spend_sample = spend_sample[spend_sample["name"].str.contains('csv.gz')]

    s = '2021/' + month_cd
    spend_sample = spend_sample[spend_sample["parent"].str.contains(s)]

    request_files_and_save_to_drive(spend_sample, cols, access_token, month_counter=month_cd, dir_path=dirname)

0it [00:00, ?it/s]


Read file 0.


/Users/julialromero/miniconda3/envs/myenv/lib/python3.9/site-packages/geopandas/geoseries.py:207: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = pd.Series(data, index=index, name=name, **kwargs)
1it [00:17, 17.07s/it]

Saved 11359 rows.

Read file 1.


/Users/julialromero/miniconda3/envs/myenv/lib/python3.9/site-packages/geopandas/geoseries.py:207: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = pd.Series(data, index=index, name=name, **kwargs)
2it [00:33, 16.84s/it]

...

Saved 11619 rows.

Read file 93.


/Users/julialromero/miniconda3/envs/myenv/lib/python3.9/site-packages/geopandas/geoseries.py:207: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = pd.Series(data, index=index, name=name, **kwargs)
94it [31:48, 18.46s/it]

Saved 11728 rows.

Read file 94.


/Users/julialromero/miniconda3/envs/myenv/lib/python3.9/site-packages/geopandas/geoseries.py:207: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = pd.Series(data, index=index, name=name, **kwargs)
95it [32:10, 20.32s/it]

Saved 11704 rows.
--------done: month 12--------


In [33]:
from mobility_processor import *

ct = CensusTractMobility(tract_data_dir='../Tracts/nyc_metro_boundaries/nyc_metro_boundaries.shp')

  STATEFP COUNTYFP TRACTCE        GEOID  NAME           NAMELSAD  MTFCC  \
0      09      003  514500  09003514500  5145  Census Tract 5145  G5020   
1      09      003  514600  09003514600  5146  Census Tract 5146  G5020   
2      09      003  514700  09003514700  5147  Census Tract 5147  G5020   
3      09      003  514800  09003514800  5148  Census Tract 5148  G5020   
4      09      003  514900  09003514900  5149  Census Tract 5149  G5020   

  FUNCSTAT    ALAND  AWATER     INTPTLAT      INTPTLON state_abbr  \
0        S  2673486   48144  +41.7772247  -072.5398572         CT   
1        S  2144936       0  +41.7692952  -072.5444530         CT   
2        S  1383464       0  +41.7698536  -072.5227784         CT   
3        S  1377342       0  +41.7708559  -072.5106023         CT   
4        S  3546060   22405  +41.7907521  -072.4835364         CT   

                                            geometry  
0  POLYGON ((-72.56218 41.78077, -72.56122 41.780...  
1  POLYGON ((-72.55995 4

In [35]:
len(ct.node_idx_map.keys())

8231

In [38]:
pd.Series(ct.node_idx_map.keys()).nunique()

8231

In [39]:
graph_obj_path = '../safegraph/graph_checkpoints/nyc_metro/checkpoint_1.pkl'
with open(graph_obj_path, 'rb') as f:
    g = pickle.load(f) # CensusTractMobility object
idx_node_map = g.get_idx_node()

/var/folders/b6/qqcnr7cj16sd1r0nn3p4f_sw0000gn/T/ipykernel_77706/2595457795.py:3: UserWarning: Unpickling a shapely <2.0 geometry object. Please save the pickle again; shapely 2.1 will not have this compatibility.
  g = pickle.load(f) # CensusTractMobility object


In [41]:
len(idx_node_map.keys())

8231

In [42]:
g.tract_data['GEOID']

0       09003514500
1       09003514600
2       09003514700
3       09003514800
4       09003514900
           ...     
8226    34029734003
8227    34013014000
8228    34013014400
8229    34013013600
8230    34013013700
Name: GEOID, Length: 8231, dtype: object